In [60]:
import tabula
import numpy as np

pdf_path = "./pdfs/2008.pdf"

# Read PDF and extract tables from pages 26 to 41
tables = tabula.read_pdf(pdf_path, pages='28-50', multiple_tables=True, stream=False)

# Store cleaned rows and rows starting with 'nan'
cleaned_rows = []
nan_rows = []

# Process tables
for table in tables:
    for row in table.itertuples(index=False):
        if row[0] == np.nan:
            nan_rows.append(row)
        else:
            cleaned_row = []
            for cell in row:
                if isinstance(cell, str):
                    cleaned_cell = ' '.join(cell.split())
                    cleaned_row.append(cleaned_cell)
                else:
                    cleaned_row.append(cell)
            cleaned_rows.append(cleaned_row)

# Write cleaned rows to a new file
output_file = "./2008.txt"
with open(output_file, 'w') as file:
    for row in cleaned_rows:
        file.write('\t'.join(map(str, row)) + '\n')

# Append the rows starting with 'nan' to the new file
with open(output_file, 'a') as file:
    file.write("#### Note: These are the rows starting with 'nan':\n")
    for row in nan_rows:
        cleaned_row = []
        for cell in row:
            if isinstance(cell, str):
                cleaned_cell = ' '.join(cell.split())
                cleaned_row.append(cleaned_cell)
            else:
                cleaned_row.append(cell)
        file.write('\t'.join(map(str, cleaned_row)) + '\n')


Got stderr: Jun 24, 2023 1:26:31 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



In [ ]:
import re
import tabula

pdf_path = "./pdfs/2005.pdf"

# Read PDF and extract tables from pages 26 to 41
tables = tabula.read_pdf(pdf_path, pages='22-34', multiple_tables=True, stream=False)

# Save tables to a text file
output_file = "./2005.txt"
with open(output_file, 'w', encoding='utf-8') as file:
    for table in tables:
        for row in table.itertuples(index=False):
            cleaned_row = []
            for cell in row:
                if isinstance(cell, str):
                    cleaned_cell = ' '.join(cell.split())
                    cleaned_row.append(cleaned_cell)
                else:
                    cleaned_row.append(cell)
            csv_contents = '\t'.join(map(str, cleaned_row)) + '\n'
            
            # Apply regex pattern to each row
            pattern = r'^(.*?),"([^"]+)",.*?,((?:\d+\.?\d*|),(?:\d+\.?\d*|),(?:\d+\.?\d*|),)'
            matches = re.findall(pattern, csv_contents, re.MULTILINE)
            
            if matches:
                # If matches are found, write them to the file
                for match in matches:
                    file.write('\t'.join(match) + '\n')
            else:
                # If no matches are found, write the original row to the file
                file.write(csv_contents)


In [ ]:
import re
###2002-2004
def convert_to_tsv(content):
    # Pattern for the headers
    header_pattern = r"Employer Name City State HRC Rating"
    
    # Pattern for the lines with company data
    company_pattern = r"^(.*\S)\s+([A-Za-z\s-]+?)\s+([A-Z]{2})\s+(\d+)$"
    
    tsv_lines = []
    removed_lines = []
    
    for line in content.split("\n"):
        if re.match(header_pattern, line):
            # Skip headers
            continue
        
        match = re.match(company_pattern, line)
        if match:
            tsv_line = "\t".join(match.groups())
            tsv_lines.append(tsv_line)
        else:
            # Collect lines that did not match
            removed_lines.append(line)
    
    tsv_content = "\n".join(tsv_lines)
    
    # Add removed lines to the end with a special note
    if removed_lines:
        tsv_content += "\n\n##Some rows did not match the pattern:\n"
        tsv_content += "\n".join(removed_lines)
    
    return tsv_content

# Read from the input file
with open("copy_paste/2005.tsv", "r", encoding="utf-8" ) as file:
    content = file.read()

# Convert the content to TSV format
tsv_content = convert_to_tsv(content)

# Write to the output file
with open("output.tsv", "w") as file:
    file.write(tsv_content)


In [ ]:
import re

with open("copy_paste/2005.tsv", "r", encoding="utf-8") as file:
    content = file.readlines()

cleaned_content = []

for line in content:
    # Remove individual 3's from the line
    line = re.sub(r'\b3\b', '', line)

    # Extract the name and the first three numbers after the string
    match = re.search(r'^(.*?)((?:\s+\d+){3})', line)
    if match:
        name = match.group(1).strip()
        numbers = match.group(2).split()
        if len(numbers) == 3:
            cleaned_line = [name] + numbers
        else:
            cleaned_line = [name, None] + numbers
        cleaned_content.append(cleaned_line)

# Fix the city names in the cleaned content
fixed_content = []
for line in cleaned_content:
    fixed_line = re.sub(r'\s+([A-Z]{2})\s+(\d+)$', r'\t\1\t\2', line[0])
    fixed_content.append(fixed_line + '\t' + '\t'.join(str(item) for item in line[1:]))

# Write the cleaned and fixed content to output.tsv
with open("output.tsv", "w", encoding="utf-8") as output_file:
    for line in fixed_content:
        output_file.write(line + '\n')


In [ ]:
import re

state_data = states

# Create a dictionary to map state names to abbreviations
state_abbrev_map = {state["state"].lower(): state["abbrev"] for state in state_data}

# Define the regex pattern
pattern = r"^(.+?)\s+((?:[A-Za-z\s]+?,\s)?[A-Za-z\s]+?)\t(\d+)\t(\d+)\t(\d+)$"

# Read input data from file
with open("output.tsv", "r", encoding="utf-8") as file:
    input_data = file.readlines()

output = "firm\tcity\tstate\tscore1\tscore2\tscore3\n"

for line in input_data:
    match = re.match(pattern, line)
    if match:
        firm = match.group(1)
        city = match.group(2)
        score1 = match.group(3)
        score2 = match.group(4)
        score3 = match.group(5)

        # Find the state abbreviation using the state name
        state_name = city.split(",")[-1].strip().lower()
        state_abbrev = state_abbrev_map.get(state_name, "N/A")

        output += f"{firm}\t{city}\t{state_abbrev}\t{score1}\t{score2}\t{score3}\n"

# Write output to file
with open("output_2.tsv", "w", encoding="utf-8") as file:
    file.write(output)


In [41]:
import pandas as pd
import numpy as np

# Read the file into a DataFrame
df = pd.read_csv("2008.tsv", sep="\t", header=None, error_bad_lines=False)

# Select only the first 7 columns and drop the rest
df = df.iloc[:, :7]

# Assigning column names to the selected 7 columns
df.columns = ["COMPANY", "CITY", "STATE", "FORTUNE", "FORBES", "AMLAW", "CEI"]

# Convert the columns 3 to 6 to string
df.iloc[:, 3:7] = df.iloc[:, 3:7].astype(str)

# Normalize values in the columns 3 to 6
df.iloc[:, 3:7] = df.iloc[:, 3:7].replace({'\.0': ''}, regex=True)

# Replace '5' and '15' with NaN
df.iloc[:, 3:7] = df.iloc[:, 3:7].replace({'5': np.nan, '15': np.nan})

# Replace 'nan' string to actual NaN value
df = df.replace({'nan': np.nan})

# Drop duplicate rows
df = df.drop_duplicates()

# Save the modified DataFrame to a new file, replacing NaN values with 'N/A'
df.to_csv("modified_2007.tsv", sep="\t", na_rep='N/A', header=True, index=None)


C:\Users\evan\AppData\Local\Temp\ipykernel_2472\2798094446.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("2006.tsv", sep="\t", header=None, error_bad_lines=False)
b'Skipping line 5: expected 17 fields, saw 18\nSkipping line 7: expected 17 fields, saw 18\nSkipping line 8: expected 17 fields, saw 18\nSkipping line 9: expected 17 fields, saw 18\nSkipping line 11: expected 17 fields, saw 18\nSkipping line 12: expected 17 fields, saw 18\nSkipping line 13: expected 17 fields, saw 18\nSkipping line 14: expected 17 fields, saw 19\nSkipping line 16: expected 17 fields, saw 18\nSkipping line 18: expected 17 fields, saw 18\nSkipping line 20: expected 17 fields, saw 18\nSkipping line 21: expected 17 fields, saw 18\nSkipping line 22: expected 17 fields, saw 18\nSkipping line 23: expected 17 fields, saw 18\nSkipping line 25: expected 17 fields, saw 18\nSkipping line 27: expected 17 f

In [ ]:
import pandas as pd

def read_csv_manual(filename, delimiter='\t', num_fields=17):
    lines_to_skip = []
    data = []
    
    with open(filename, 'r') as file:
        for line_num, line in enumerate(file):
            fields = line.strip().split(delimiter)
            if len(fields) != num_fields:
                print(f"Skipping line {line_num + 1}: expected {num_fields} fields, saw {len(fields)}")
                lines_to_skip.append(line_num + 1)
            else:
                data.append(fields)
    
    df = pd.DataFrame(data)
    return df, lines_to_skip

# Usage
filename = "2006.tsv"
df, skipped_lines = read_csv_manual(filename)

# Save DataFrame to a CSV file
df.to_csv("output.csv", index=False)

print(df.head())
print("Skipped lines:", skipped_lines)


In [48]:
# Open the file for reading
with open('2006.tsv', 'r') as file:
    lines = file.readlines()

# Use a dictionary to store firm names and their longest rows
unique_firms = {}
for line in lines:
    row = line.strip().split('\t')
    firm_name = row[0]
    row_length = len(row)
    
    if firm_name not in unique_firms or row_length > len(unique_firms[firm_name]):
        unique_firms[firm_name] = row

# Write the longest rows to the output file
with open('output.tsv', 'w') as file:
    for row in unique_firms.values():
        file.write('\t'.join(row) + '\n')


In [49]:
# Open the input file for reading
with open('output.tsv', 'r') as file:
    lines = file.readlines()

# Extract the first seven values from each row
values = [line.strip().split('\t')[:7] for line in lines]

# Write the extracted values to the output2 file
with open('output2.tsv', 'w') as file:
    for row in values:
        file.write('\t'.join(row) + '\n')


In [52]:
import csv

input_file = 'output2.tsv'
output_file = 'output2_shifted.tsv'

# Read the input TSV file
rows = []
with open(input_file, 'r', newline='') as file:
    reader = csv.reader(file, delimiter='\t')
    rows = list(reader)

# Determine the maximum length of rows
max_length = max(len(row) for row in rows)

# Adjust the rows to have a length of 7 and shift the last number
for row in rows:
    while len(row) < 7:
        row.insert(3, 'N/A')
    last_number = None
    for i in range(len(row) - 1, -1, -1):
        if row[i] != '':
            last_number = row[i]
            row[i] = 'N/A'
            for j in range(i - 1, -1, -1):
                if row[j] == '':
                    row[j] = 'N/A'
                else:
                    break
            break
    if last_number is not None:
        row[6] = last_number

# Write the updated data to a new TSV file
with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file, delimiter='\t')
    writer.writerows(rows)

print(f"Updated data has been written to {output_file}.")


Updated data has been written to output2_shifted.tsv.


In [67]:
import csv
import re

# Read 2006.tsv and create a dictionary of company names and cities
company_city_2006 = {}
with open('./copy_paste/2006.tsv', 'r') as file_2006:
    reader_2006 = csv.reader(file_2006, delimiter='\t')
    next(reader_2006)  # Skip the header row
    for row in reader_2006:
        if len(row) >= 2:
            company = row[0]
            city_state = row[1].replace(',', '\t')
            company_city_2006[company] = city_state

# Open 2008.tsv and output.tsv files
with open('./2008.tsv', 'r') as file_2008, open('./output33.tsv', 'w') as outfile:
    reader_2008 = csv.reader(file_2008, delimiter='\t')
    writer = csv.writer(outfile, delimiter='\t')

    # Regex pattern to match company, city and state in the string
    pattern = re.compile(r'^(.*?)\t(.*?),\s*(\w{2})\t')

    # Iterate over 2008.tsv rows
    for row in reader_2008:
        # Only process rows with at least 2 columns
        if len(row) >= 3:
            match = pattern.match(row[0])
            if match:
                company, city, state = match.groups()
                if company in company_city_2006:
                    city_state_2006 = company_city_2006[company]
                    # Replace the city and state with the values from 2006.tsv
                    row[0] = pattern.sub(f'{company}\t{city_state_2006}\t', row[0])
                else:
                    row[0] = '\n' + row[0]

        writer.writerow(row)


In [70]:
import csv

filename = '2008.tsv'
output_filename = 'out2.tsv'

with open(filename, 'r', encoding='utf-8', newline='') as file:
    reader = csv.reader(file, delimiter='\t')
    header = next(reader)  # Read the header row

    # Find the indices of the desired columns
    column_indices = [header.index(column) for column in ['COMPANY', 'CITY', 'STATE', 'FORTUNE', 'FORBES', 'AMLAW', '2008_CEI', '2006_CEI']]
    
    with open(output_filename, 'w', encoding='utf-8', newline='') as output_file:
        writer = csv.writer(output_file, delimiter='\t')
        writer.writerow(header + ['ERROR_NOTE'])  # Write the header row with the error note

        for row in reader:
            data = []

            # Extract the values for the desired columns
            for index in column_indices:
                if index < len(row):
                    data.append(row[index])
                else:
                    data.append('')
            
            if len(row) < len(header):
                data.append('##ERROR_HERE_123!')
            
            writer.writerow(data)


In [84]:
import csv

def parse_2008_tsv():
    with open('2008.tsv', 'r') as file:
        reader = csv.reader(file, delimiter='\t')
        rows = list(reader)

    with open('out2.tsv', 'w', newline='') as file:
        writer = csv.writer(file, delimiter='\t')

        for row in rows:
            state_abbrev = row[2]
            last_item = row[-1]
            if state_abbrev != 'nan' and last_item.endswith('nan') and any(isinstance(item, float) and item.is_integer() for item in row[3:-1]):
                row = row[:7]  # Keep the first 7 items (indices 0 to 6)
            else:
                row = row[:8]  # Keep the first 8 items (indices 0 to 7)
            writer.writerow(row)

parse_2008_tsv()
